In [1]:
# DSSM is a two-tower neural network architecture for learning semantic representations
# of users and items (or queries and documents) in a shared embedding space.
# We will use PyTorch to build a DSSM that matches users to items.



In [1]:
# ----------------------
# 2. IMPORTS AND SETUP
# ----------------------
import os
import os.path
import requests
import zipfile
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

/Users/kulyaskin_mikhail/ITMO/RecSys/RecSys/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# ----------------------
# 3. DOWNLOAD AND LOAD DATA
# ----------------------
data_path = os.environ.get("DATA_PATH")

if data_path is None:
    data_path = "/Users/kulyaskin_mikhail/ITMO/RecSys/data/data_original"  # ваш путь к данным до папки data_original включительно

In [10]:
# ----------------------
# 4. DATA PREPROCESSING
# ----------------------
interactions_df = pd.read_csv(os.path.join(data_path, "interactions.csv"))
users_df = pd.read_csv(os.path.join(data_path, "users.csv"))
items_df = pd.read_csv(os.path.join(data_path, "items.csv"))

In [13]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом,
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,sex_Ж,sex_М,kids_flg_0,kids_flg_1
0,973171,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,True
1,962099,True,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False
2,1047345,False,False,False,True,False,False,False,False,False,True,False,False,True,False,True,False
3,721985,False,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False
4,704055,False,False,True,False,False,False,False,False,False,False,True,False,True,False,True,False


In [14]:
item_cat_feats = ['content_type', 'release_year',
                  'for_kids', 'age_rating',
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1)

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_1897.0,release_year_1916.0,release_year_1917.0,release_year_1918.0,release_year_1920.0,release_year_1921.0,release_year_1922.0,...,directors_Яннике Систад Якобсен,directors_Янус Мец,directors_Ярив Хоровиц,directors_Ярон Зильберман,directors_Ярополк Лапшин,directors_Ярослав Лупий,"directors_Ярроу Чейни, Скотт Моужер",directors_Ясина Сезар,directors_Ясуоми Умэцу,directors_сения Завьялова
0,10711,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2508,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,10716,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,7868,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,16268,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [15]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


In [16]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6901


In [17]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6897


In [18]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72.0,10616,3944
1,699317,1659,2021-05-29,8317,100.0,42131,675
6,1016458,354,2021-08-14,1672,25.0,61024,139
7,884009,693,2021-08-04,703,14.0,53150,279
14,5324,8437,2021-04-18,6598,92.0,310,3485


In [19]:
interactions_vec = np.zeros((interactions_df.uid.nunique(),
                             interactions_df.iid.nunique()))

for user_id, item_id in zip(interactions_df.uid, interactions_df.iid):
    interactions_vec[user_id, item_id] += 1


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [20]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

6897
6897
65974
65974
set()


In [21]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

In [22]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

In [23]:
N_FACTORS = 32

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], )
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 32
ITEM_MODEL_SHAPE: (8813,)
USER_META_MODEL_SHAPE: (16,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [27]:
class ItemModel(nn.Module):
    def __init__(self, n_factors=N_FACTORS, nhead=8, num_layers=2, dropout=0.2):
        super(ItemModel, self).__init__()
        self.input_proj = nn.Linear(ITEM_MODEL_SHAPE[0], n_factors)
        self.input_norm = nn.LayerNorm(n_factors)
        self.input_dropout = nn.Dropout(dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=n_factors, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.output_norm = nn.LayerNorm(n_factors)
        self.output_dropout = nn.Dropout(dropout)
        self.output_proj = nn.Linear(n_factors, n_factors)
        
    def forward(self, x):
        # Проекция входных признаков до размера n_factors
        x = self.input_proj(x)
        x = self.input_norm(x)
        x = self.input_dropout(x)
        
        # Добавляем фиктивную размерность последовательности (sequence length = 1)
        x = x.unsqueeze(1)
        # Трансформер ожидает формат [seq_len, batch_size, n_factors]
        x = x.transpose(0, 1)
        # Применяем Transformer Encoder
        x = self.transformer_encoder(x)
        # Возвращаем в формат [batch_size, n_factors]
        x = x.transpose(0, 1).squeeze(1)
        
        x = self.output_norm(x)
        x = self.output_dropout(x)
        # Финальная проекция
        x = self.output_proj(x)
        return x

# Define the user model
class UserModel(nn.Module):
    def __init__(self, n_factors=N_FACTORS, nhead=8, num_layers=2, dropout=0.2):
        super(UserModel, self).__init__()
        # Проекция метаданных пользователя
        self.meta_proj = nn.Linear(USER_META_MODEL_SHAPE[0], n_factors)
        self.meta_norm = nn.LayerNorm(n_factors)
        self.meta_dropout = nn.Dropout(dropout)
        
        # Проекция взаимодействий пользователя
        self.interaction_proj = nn.Linear(USER_INTERACTION_MODEL_SHAPE[0], n_factors)
        self.interaction_norm = nn.LayerNorm(n_factors)
        self.interaction_dropout = nn.Dropout(dropout)
        
        # Трансформеры для метаданных и взаимодействий
        encoder_layer_meta = nn.TransformerEncoderLayer(d_model=n_factors, nhead=nhead, dropout=dropout)
        self.transformer_encoder_meta = nn.TransformerEncoder(encoder_layer_meta, num_layers=num_layers)
        
        encoder_layer_inter = nn.TransformerEncoderLayer(d_model=n_factors, nhead=nhead, dropout=dropout)
        self.transformer_encoder_inter = nn.TransformerEncoder(encoder_layer_inter, num_layers=num_layers)
        
        # Нормализация и Dropout для выходных представлений
        self.meta_out_norm = nn.LayerNorm(n_factors)
        self.inter_out_norm = nn.LayerNorm(n_factors)
        
        # Финальная проекция объединенных представлений
        self.final_dropout = nn.Dropout(dropout)
        self.output_proj = nn.Linear(n_factors * 2, n_factors)
        
    def forward(self, meta, interaction):
        # Проекция метаданных
        meta = self.meta_proj(meta)
        meta = self.meta_norm(meta)
        meta = self.meta_dropout(meta)
        
        # Добавляем фиктивную размерность последовательности
        meta = meta.unsqueeze(1)
        # Трансформер ожидает формат [seq_len, batch_size, n_factors]
        meta = meta.transpose(0, 1)
        # Применяем Transformer Encoder
        meta = self.transformer_encoder_meta(meta)
        # Возвращаем в формат [batch_size, n_factors]
        meta = meta.transpose(0, 1).squeeze(1)
        meta = self.meta_out_norm(meta)
        
        # Аналогично для взаимодействий
        interaction = self.interaction_proj(interaction)
        interaction = self.interaction_norm(interaction)
        interaction = self.interaction_dropout(interaction)
        
        interaction = interaction.unsqueeze(1)
        interaction = interaction.transpose(0, 1)
        interaction = self.transformer_encoder_inter(interaction)
        interaction = interaction.transpose(0, 1).squeeze(1)
        interaction = self.inter_out_norm(interaction)
        
        # Объединяем представления и применяем финальную проекцию
        x = torch.cat([meta, interaction], dim=1)
        x = self.final_dropout(x)
        x = self.output_proj(x)
        return x

In [28]:
# изменяем loss на triplet loss с расчетом косинусной близости
triplet_loss = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y),
    margin=0.4
)

In [29]:
# Define the dataset
class RecSysDataset(Dataset):
    def __init__(self, items, users, interactions):
        self.items = items
        self.users = users
        self.interactions = interactions

    def __len__(self):
        return self.interactions.shape[0]

    def __getitem__(self, idx):
        uid = idx
        pos_i = np.random.choice(range(self.interactions.shape[1]), p=self.interactions[uid])
        neg_i = np.random.choice(range(self.interactions.shape[1]))
        uid_meta = self.users.iloc[uid].values
        uid_interaction = self.interactions[uid]
        pos = self.items.iloc[pos_i].values
        neg = self.items.iloc[neg_i].values
        return torch.tensor(uid_meta, dtype=torch.float32), torch.tensor(uid_interaction, dtype=torch.float32), torch.tensor(pos, dtype=torch.float32), torch.tensor(neg, dtype=torch.float32)


In [30]:
# Initialize the models, optimizer, and dataset
i2v = ItemModel()
u2v = UserModel()
optimizer = optim.Adam(list(i2v.parameters()) + list(u2v.parameters()), lr=0.001)
dataset = RecSysDataset(items=items_ohe_df.drop(["item_id"], axis=1), users=users_ohe_df.drop(["user_id"], axis=1), interactions=interactions_vec)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [32]:
import logging

In [33]:

# Set up logging
logging.basicConfig(filename='training.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Training loop
for epoch in range(2):
    for batch in dataloader:
        uid_meta, uid_interaction, pos, neg = batch
        optimizer.zero_grad()
        anchor = u2v(uid_meta, uid_interaction)
        positive = i2v(pos)
        negative = i2v(neg)
        loss = triplet_loss(anchor, positive, negative)
        loss.backward()
        optimizer.step()

        # Log the loss for each batch
        print(f"Epoch {epoch + 1}, Batch Loss: {loss.item()}")

    # Log the epoch loss
    print(f"Epoch {epoch + 1}, Epoch Loss: {loss.item()}")

Epoch 1, Batch Loss: 0.4215570390224457
Epoch 1, Batch Loss: 0.3735736608505249
Epoch 1, Batch Loss: 0.3773857355117798
Epoch 1, Batch Loss: 0.38890400528907776
Epoch 1, Batch Loss: 0.36707454919815063
Epoch 1, Batch Loss: 0.43003368377685547
Epoch 1, Batch Loss: 0.39561575651168823
Epoch 1, Batch Loss: 0.44626086950302124
Epoch 1, Batch Loss: 0.39964061975479126
Epoch 1, Batch Loss: 0.38553956151008606
Epoch 1, Batch Loss: 0.4067000448703766
Epoch 1, Batch Loss: 0.40201669931411743
Epoch 1, Batch Loss: 0.3312397599220276
Epoch 1, Batch Loss: 0.3503345847129822
Epoch 1, Batch Loss: 0.33818185329437256
Epoch 1, Batch Loss: 0.3728874921798706
Epoch 1, Batch Loss: 0.36076509952545166
Epoch 1, Batch Loss: 0.29485899209976196
Epoch 1, Batch Loss: 0.28871339559555054
Epoch 1, Batch Loss: 0.3253413438796997
Epoch 1, Batch Loss: 0.3198901116847992
Epoch 1, Batch Loss: 0.33764833211898804
Epoch 1, Batch Loss: 0.3641596734523773
Epoch 1, Batch Loss: 0.253708153963089
Epoch 1, Batch Loss: 0.40646

In [34]:
# Inference:
# Get user and item features
rand_uid = np.random.choice(list(users_ohe_df.index))
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid].values
user_interaction_vec = interactions_vec[rand_uid]
rand_iid = np.random.choice(list(items_ohe_df.index))
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid].values

# Convert to PyTorch tensors
user_meta_feats = torch.tensor(user_meta_feats, dtype=torch.float32)
user_interaction_vec = torch.tensor(user_interaction_vec, dtype=torch.float32)
item_feats = torch.tensor(item_feats, dtype=torch.float32)

# Get user and item embeddings
user_vec = u2v(user_meta_feats.unsqueeze(0), user_interaction_vec.unsqueeze(0))
item_vec = i2v(item_feats.unsqueeze(0))

In [35]:
# Inference:
# Get user and item features
rand_uid = np.random.choice(list(users_ohe_df.index))
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid].values
user_interaction_vec = interactions_vec[rand_uid]
rand_iid = np.random.choice(list(items_ohe_df.index))
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid].values

# Convert to PyTorch tensors
user_meta_feats = torch.tensor(user_meta_feats, dtype=torch.float32)
user_interaction_vec = torch.tensor(user_interaction_vec, dtype=torch.float32)
item_feats = torch.tensor(item_feats, dtype=torch.float32)

# Get user and item embeddings
user_vec = u2v(user_meta_feats.unsqueeze(0), user_interaction_vec.unsqueeze(0))
item_vec = i2v(item_feats.unsqueeze(0))

# Calculate distance
distance = torch.dist(user_vec, item_vec)

# Print distance
print(f"Distance between user {rand_uid} and item {rand_iid}: {distance.item()}")

# Get top 10 recommendations for all users
users_meta_feats = torch.tensor(users_ohe_df.drop(["user_id"], axis=1).values, dtype=torch.float32)
users_interaction_vec = torch.tensor(interactions_vec, dtype=torch.float32)
items_feats = torch.tensor(items_ohe_df.drop(["item_id"], axis=1).values, dtype=torch.float32)

users_vec = u2v(users_meta_feats, users_interaction_vec)
items_vecs = i2v(items_feats)

dists = torch.cdist(users_vec, items_vecs)

top10_iids = torch.argsort(dists, dim=1)[:, :10]

top10_iids_item = [iid_to_item_id[iid.item()] for iid in top10_iids.reshape(-1)]
top10_iids_item = np.array(top10_iids_item).reshape(top10_iids.shape)

df_dssm = pd.DataFrame({'user_id': [uid_to_user_id[uid] for uid in np.arange(top10_iids_item.shape[0])]})
df_dssm['item_id'] = list(top10_iids_item)
df_dssm = df_dssm.explode('item_id')
df_dssm['rank'] = df_dssm.groupby('user_id').cumcount() + 1
df_dssm = df_dssm.groupby('user_id').agg({'item_id': list}).reset_index()

Distance between user 1868 and item 5721: 15.041807174682617


In [36]:
df_dssm

,user_id,item_id
0,2,"[6809, 11237, 12173, 16166, 9342, 10761, 9728,..."
1,21,"[6809, 11237, 10761, 12173, 9342, 849, 5693, 9..."
2,53,"[6809, 10761, 11237, 9728, 9342, 16166, 12173,..."
3,60,"[6809, 11237, 10761, 12173, 9728, 9342, 5693, ..."
4,81,"[6809, 9728, 11237, 4621, 849, 10761, 1844, 16..."
...,...,...
65969,1097486,"[6809, 11237, 9728, 9342, 12173, 849, 1844, 10..."
65970,1097489,"[6809, 11237, 10761, 12173, 9342, 5693, 849, 9..."
65971,1097508,"[6809, 11237, 10761, 12173, 9342, 16166, 9728,..."
65972,1097513,"[6809, 11237, 12173, 10761, 849, 9342, 9728, 5..."


Проделаны следующие изменения/эксперименты:
- замена MLM слоев на энкодер блоки трансформера для улучшение моделирования зависимостей
- замена триплет loss на триплет loss с расчетом косинусной близости для более робастного сравнения векторов 
- добавил Dropout и Layer Norm слои для уменьшения переобучения

In [25]:
# # сохранение векторов для сервиса
# import pickle
# import hnswlib

# def save_dssm_for_service(item_model, user_model, output_dir, 
#                          items_df, users_df, interactions_vec):
#     """
#     Сохраняет эмбеддинги и маппинги DSSM модели для использования в сервисе
    
#     Args:
#         item_model: Обученная модель ItemModel
#         user_model: Обученная модель UserModel
#         output_dir: Директория для сохранения файлов
#         items_df: DataFrame с данными предметов
#         users_df: DataFrame с данными пользователей
#         interactions_vec: Матрица взаимодействий пользователь-предмет
#     """
#     import os
    
#     # Создаем директорию, если она не существует
#     os.makedirs(output_dir, exist_ok=True)
    
#     print(f"Сохранение модели DSSM в директорию {output_dir}...")
    
#     # 1. Вычисляем эмбеддинги предметов
#     item_model.eval()
#     with torch.no_grad():
#         items_feats = torch.tensor(items_df.drop(["item_id"], axis=1).values, dtype=torch.float32)
#         item_embeddings = item_model(items_feats).cpu().numpy()
    
#     # 2. Вычисляем эмбеддинги пользователей
#     user_model.eval()
#     with torch.no_grad():
#         users_meta = torch.tensor(users_df.drop(["user_id"], axis=1).values, dtype=torch.float32)
#         users_interactions = torch.tensor(interactions_vec, dtype=torch.float32)
#         user_embeddings = user_model(users_meta, users_interactions).cpu().numpy()
    
#     # 3. Создаем маппинги ID -> индексы и наоборот
#     user_id_to_idx = {user_id: idx for idx, user_id in enumerate(users_df.user_id.values)}
#     idx_to_user_id = {idx: user_id for user_id, idx in user_id_to_idx.items()}
    
#     item_id_to_idx = {item_id: idx for idx, item_id in enumerate(items_df.item_id.values)}
#     idx_to_item_id = {idx: item_id for item_id, idx in item_id_to_idx.items()}
    
#     mappings = {
#         "user_id_to_idx": user_id_to_idx,
#         "idx_to_user_id": idx_to_user_id,
#         "item_id_to_idx": item_id_to_idx,
#         "idx_to_item_id": idx_to_item_id
#     }
    
#     # 4. Сохраняем эмбеддинги
#     np.save(os.path.join(output_dir, "user_embeddings.npy"), user_embeddings)
#     np.save(os.path.join(output_dir, "item_embeddings.npy"), item_embeddings)
    
#     # 5. Сохраняем маппинги
#     with open(os.path.join(output_dir, "id_mappings.pkl"), 'wb') as f:
#         pickle.dump(mappings, f)
    
#     # 6. Создаем и сохраняем HNSW индекс
#     dim = item_embeddings.shape[1]
#     num_elements = item_embeddings.shape[0]
    
#     # Инициализируем индекс с косинусным расстоянием
#     index = hnswlib.Index(space='cosine', dim=dim)
#     index.init_index(max_elements=num_elements, ef_construction=200, M=50)
    
#     # Добавляем эмбеддинги предметов в индекс
#     index.add_items(item_embeddings)
    
#     # Устанавливаем параметр ef для поиска
#     index.set_ef(30)
    
#     # Сохраняем индекс
#     index.save_index(os.path.join(output_dir, "hnsw_index.bin"))
    
#     # Сохраняем конфигурацию индекса
#     config_params = {
#             "hnsw_params": {
#                 "space": "cosine",
#                 "dim": dim,
#                 "efC": 200,
#                 "efS": 30,
#                 "M": 50
#             },
#         }
    
#     with open(os.path.join(output_dir, "config.pkl"), 'wb') as f:
#         pickle.dump(config_params, f)
    
#     print("Сохранение завершено. Сохранены следующие файлы:")
#     print(f"- {os.path.join(output_dir, 'user_embeddings.npy')}")
#     print(f"- {os.path.join(output_dir, 'item_embeddings.npy')}")
#     print(f"- {os.path.join(output_dir, 'id_mappings.pkl')}")
#     print(f"- {os.path.join(output_dir, 'hnsw_index.bin')}")
#     print(f"- {os.path.join(output_dir, 'config.pkl')}")
    
#     return {
#         "user_embeddings_path": os.path.join(output_dir, "user_embeddings.npy"),
#         "item_embeddings_path": os.path.join(output_dir, "item_embeddings.npy"),
#         "id_mappings_path": os.path.join(output_dir, "id_mappings.pkl"),
#         "hnsw_index_path": os.path.join(output_dir, "hnsw_index.bin"),
#         "hnsw_mappings_path": os.path.join(output_dir, "config.pkl")
#     }

In [ ]:
# output_dir = "/Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm"
# save_dssm_for_service(i2v, u2v, output_dir, 
#                       items_ohe_df.reset_index(drop=True), 
#                       users_ohe_df.reset_index(drop=True), 
#                       interactions_vec)

Сохранение модели DSSM в директорию /Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm...
Сохранение завершено. Сохранены следующие файлы:
- /Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/user_embeddings.npy
- /Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/item_embeddings.npy
- /Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/id_mappings.pkl
- /Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/hnsw_index.bin
- /Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/config.pkl


{'user_embeddings_path': '/Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/user_embeddings.npy',
 'item_embeddings_path': '/Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/item_embeddings.npy',
 'id_mappings_path': '/Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/id_mappings.pkl',
 'hnsw_index_path': '/Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/hnsw_index.bin',
 'hnsw_mappings_path': '/Users/kulyaskin_mikhail/ITMO/RecSys/data/dssm/config.pkl'}